### 7. Use Linear Regression to analyze the correlation among wine quality and several key factors of wines, such as pH and density. 

In [1]:
# imports:

import pandas as pd
import numpy as np
import sklearn 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [4]:
# 1. Read the data:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
val = pd.read_csv('val.csv')

target_column = 'quality'

x_train = train.drop(columns=[target_column])
print(x_train)
y_train = train[target_column]
print(y_train)

     Unnamed: 0  fixed_acidity  volatile_acidity  ...        pH  sulphates   alcohol
0           233       2.626785          0.721110  ...  1.860108   0.707107  3.316625
1           241       3.464102          0.616441  ...  1.772005   0.842615  3.301515
2           268       2.626785          0.734847  ...  1.920937   0.953939  3.065942
3           299       2.626785          0.818535  ...  1.918333   0.842615  3.098387
4          1104       2.828427          0.692820  ...  1.811077   0.812404  3.521363
..          ...            ...               ...  ...       ...        ...       ...
954         699       3.435113          0.616441  ...  1.763519   0.787401  3.286335
955        1434       3.193744          0.734847  ...  1.783255   0.877496  3.000000
956         601       3.633180          0.678233  ...  1.760682   0.748331  3.000000
957         693       3.000000          0.692820  ...  1.822087   0.787401  3.065942
958         582       3.420526          0.700000  ...  1.786057  

In [7]:
# 2. Write a function to train linear regressor using training data.
# First, compute quadratic features from input features (encouraged to try linear/cubic features). 
# Scipy:
degree = 2 # set to 2 to use quadratic features
poly = PolynomialFeatures(degree=degree, include_bias=False)
x_train_poly = poly.fit_transform(x_train)
x_val_poly = poly.transform(val.drop(columns=[target_column]))
model = LinearRegression()
model.fit(x_train_poly, y_train)

LinearRegression()

In [9]:
# 3. To evaluate your model, compute Mean Squared Error (MSE) on training data and validation data
from sklearn.metrics import mean_squared_error
y_train_pred = model.predict(x_train_poly)
y_val_pred = model.predict(x_val_poly)
train_mse = mean_squared_error(y_train, y_train_pred)
val_mse = mean_squared_error(val[target_column], y_val_pred)
print(f"Train MSE: {train_mse}")
print(f"Validation MSE: {val_mse}")

Train MSE: 0.33069268586180933
Validation MSE: 0.4146219002573809


In [ ]:
# 4. Notice the low errors on train data and high errors on validation. 
# This issue is called overfitting.
    # Models learn certain feature-target correlations that only exist in training data that does not generalize to test data. 
# To mitigate this, use L2 regularization (Ridge regression) to penalize large weights.

from sklearn.linear_model import Ridge
alpha = 0 # controls intensity of L2 regularization
model = Ridge(alpha=alpha)
model.fit(x_train_poly, y_train)
# try different alphas, observe changes in validation error. 

In [ ]:
# 5. Based on your observations of validation errors, choose the optimal L2 regularization intensity. Repoprt the performance of your best model on test data in test.csv. 